In [1]:
# Importar as bibliotecas necessárias
import pandas as pd
import numpy as np
import networkx as nx
from multiprocessing import Pool, cpu_count
import time

In [2]:
# Função para processar cada subconjunto de votos
def processar_votacoes(df_votos, tipo_voto, total_votacoes, idx_total_votacao):
    print(f"[{tipo_voto}] Iniciando processamento de {len(df_votos)} votações...")
    G_local = nx.Graph()  # Grafo local para o processo
    votacoes_unicas = df_votos['id_votacao'].unique()
    total_comparacoes_acumuladas = 0

    # Processar votações
    for idx, id_votacao in enumerate(votacoes_unicas):
        start_votacao = time.time()  # Marca o início da votação atual

        # Filtrar os dados para essa votação
        votacao_atual = df_votos[df_votos['id_votacao'] == id_votacao]
        deputados_na_votacao = votacao_atual['id_deputado'].values
        total_deputados_na_votacao = len(deputados_na_votacao)

        # Adicionar nós (deputados) ao grafo com atributos
        for deputado in deputados_na_votacao:
            if deputado not in G_local.nodes:
                nome = votacao_atual[votacao_atual['id_deputado'] == deputado]['nome'].values[0]
                partido = votacao_atual[votacao_atual['id_deputado'] == deputado]['sigla_partido'].values[0]
                estado = votacao_atual[votacao_atual['id_deputado'] == deputado]['sigla_uf'].values[0]
                
                # Adiciona o deputado ao grafo com atributos de nome, partido e estado
                G_local.add_node(deputado, nome=nome, partido=partido, estado=estado)

        # Calcular o total de comparações possíveis
        total_comparacoes_possiveis = (total_deputados_na_votacao * (total_deputados_na_votacao - 1)) // 2
        comparacoes_feitas = 0

        # Adicionar arestas para deputados que votaram da mesma forma
        for i in range(total_deputados_na_votacao):
            deputado_1 = deputados_na_votacao[i]
            for j in range(i + 1, total_deputados_na_votacao):  # Apenas comparam deputado_1 != deputado_2
                deputado_2 = deputados_na_votacao[j]
                if deputado_1 != deputado_2:  # Verifica se não é um self-loop
                    comparacoes_feitas += 1

                    # Atualizar as arestas no grafo local
                    if not G_local.has_edge(deputado_1, deputado_2):  # Adicionar aresta apenas se ainda não existe
                        G_local.add_edge(deputado_1, deputado_2, weight=1)
                    else:
                        G_local[deputado_1][deputado_2]['weight'] += 1

        # Atualizações após processar a votação atual
        total_comparacoes_acumuladas += comparacoes_feitas
        tempo_votacao = time.time() - start_votacao  # Calcula o tempo gasto na votação atual
        print(f"[{tipo_voto}] Votação {idx_total_votacao + idx + 1}/{total_votacoes} (ID: {id_votacao}) concluída. "
              f"Total de comparações feitas: {total_comparacoes_acumuladas}")
    
    print(f"[{tipo_voto}] Concluído processamento de {len(votacoes_unicas)} votações.")
    return G_local

In [3]:
# Função para exportar o grafo com as informações dos deputados para um arquivo .gml
def exportar_grafo_gml(G, filename="grafo_deputados.gml"):
    nx.write_gml(G, filename)
    print(f"Grafo exportado com sucesso para {filename}")

In [4]:
# Carregar as tabelas
votacoes_polarizadas = pd.read_csv('../data/votacoes_polarizadas.csv')
votacao_parlamentar = pd.read_csv('../data/dados abertos/votacao_parlamentar.csv')

In [8]:
# Realizar a junção das tabelas mantendo apenas as votações presentes na tabela "votacoes_polarizadas"
analise_votacao = pd.merge(votacoes_polarizadas, 
                           votacao_parlamentar[['id_votacao', 'voto', 'id_deputado', 'nome', 'sigla_partido', 'sigla_uf']], 
                           on='id_votacao', 
                           how='left')

# Exibir as primeiras linhas do dataframe resultante
analise_votacao

,id_proposicao,ano_proposicao,data_proposicao,sigla,tipo,tema,id_votacao,data_votacao,sigla_orgao,aprovacao,...,voto_outro,ano_votacao,total_votos,percentual_voto_sim,percentual_voto_sim_intervalo,voto,id_deputado,nome,sigla_partido,sigla_uf
0,351764,2007,2007-05-09,RDF,Redação Final,NaN,345069121.0,2007-06-26,PLEN,1.0,...,2.0,2007,309.0,0.566343,51-60%,Não,141335.0,Beto Faro,PT,PA
1,351764,2007,2007-05-09,RDF,Redação Final,NaN,345069121.0,2007-06-26,PLEN,1.0,...,2.0,2007,309.0,0.566343,51-60%,Sim,141503.0,Jô Moraes,PCdoB,MG
2,351764,2007,2007-05-09,RDF,Redação Final,NaN,345069121.0,2007-06-26,PLEN,1.0,...,2.0,2007,309.0,0.566343,51-60%,Sim,141519.0,Paulo Piau,PMDB,MG
3,351764,2007,2007-05-09,RDF,Redação Final,NaN,345069121.0,2007-06-26,PLEN,1.0,...,2.0,2007,309.0,0.566343,51-60%,Sim,141416.0,Edgar Moury,PMDB,PE
4,351764,2007,2007-05-09,RDF,Redação Final,NaN,345069121.0,2007-06-26,PLEN,1.0,...,2.0,2007,309.0,0.566343,51-60%,Não,74162.0,Iriny Lopes,PT,ES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4846513,2366774,2023,2023-05-31,RPD,Requerimento de Encerramento da Discussão e do...,NaN,234549350.0,2023-05-31,PLEN,0.0,...,3.0,2023,428.0,0.425234,41-50%,Não,160610.0,Valmir Assunção,PT,BA
4846514,2366774,2023,2023-05-31,RPD,Requerimento de Encerramento da Discussão e do...,NaN,234549350.0,2023-05-31,PLEN,0.0,...,3.0,2023,428.0,0.425234,41-50%,Não,91228.0,Waldemar Oliveira,AVANTE,PE
4846515,2366774,2023,2023-05-31,RPD,Requerimento de Encerramento da Discussão e do...,NaN,234549350.0,2023-05-31,PLEN,0.0,...,3.0,2023,428.0,0.425234,41-50%,Não,141411.0,Dagoberto Nogueira,PSDB,MS
4846516,2366774,2023,2023-05-31,RPD,Requerimento de Encerramento da Discussão e do...,NaN,234549350.0,2023-05-31,PLEN,0.0,...,3.0,2023,428.0,0.425234,41-50%,Não,137070.0,Vicentinho Júnior,PP,TO


In [9]:
# Filtrar as votações apenas do ano selecionado
ano = 2022
analise_votacao_filt = analise_votacao[analise_votacao['ano_votacao'] == ano]

In [10]:
analise_votacao_filt

,id_proposicao,ano_proposicao,data_proposicao,sigla,tipo,tema,id_votacao,data_votacao,sigla_orgao,aprovacao,...,voto_outro,ano_votacao,total_votos,percentual_voto_sim,percentual_voto_sim_intervalo,voto,id_deputado,nome,sigla_partido,sigla_uf
441189,2325754,2022,2022-06-01,DTQ,Destaque,NaN,2.309053e+09,2022-06-01,PLEN,1.0,...,1.0,2022,360.0,0.572222,51-60%,Não,178832.0,Leandre,PSD,PR
441190,2325754,2022,2022-06-01,DTQ,Destaque,NaN,2.309053e+09,2022-06-01,PLEN,1.0,...,1.0,2022,360.0,0.572222,51-60%,Sim,178962.0,Carlos Gomes,REPUBLICANOS,RS
441191,2325754,2022,2022-06-01,DTQ,Destaque,NaN,2.309053e+09,2022-06-01,PLEN,1.0,...,1.0,2022,360.0,0.572222,51-60%,Sim,215043.0,Josivaldo JP,PSD,MA
441192,2325754,2022,2022-06-01,DTQ,Destaque,NaN,2.309053e+09,2022-06-01,PLEN,1.0,...,1.0,2022,360.0,0.572222,51-60%,Não,204525.0,Rosana Valle,PL,SP
441193,2325754,2022,2022-06-01,DTQ,Destaque,NaN,2.309053e+09,2022-06-01,PLEN,1.0,...,1.0,2022,360.0,0.572222,51-60%,Sim,204513.0,Guiga Peixoto,PSC,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3817718,1672631,2015,2015-08-20,REQ,Requerimento de Constituição de Comissão Espec...,NaN,1.546017e+07,2022-02-24,PLEN,0.0,...,6.0,2022,392.0,0.413265,41-50%,Não,74270.0,Gilberto Nascimento,PSC,SP
3817719,1672631,2015,2015-08-20,REQ,Requerimento de Constituição de Comissão Espec...,NaN,1.546017e+07,2022-02-24,PLEN,0.0,...,6.0,2022,392.0,0.413265,41-50%,Sim,204372.0,Julio Cesar Ribeiro,REPUBLICANOS,DF
3817720,1672631,2015,2015-08-20,REQ,Requerimento de Constituição de Comissão Espec...,NaN,1.546017e+07,2022-02-24,PLEN,0.0,...,6.0,2022,392.0,0.413265,41-50%,Sim,178924.0,Capitão Fábio Abreu,PL,PI
3817721,1672631,2015,2015-08-20,REQ,Requerimento de Constituição de Comissão Espec...,NaN,1.546017e+07,2022-02-24,PLEN,0.0,...,6.0,2022,392.0,0.413265,41-50%,Sim,204565.0,Professora Dayane Pimentel,UNIÃO,BA


In [13]:
analise_votacao_filt['id_deputado'].nunique()

554

In [14]:
# Filtrar os dados para pegar apenas os primeiros deputados
n_deputados = 100

In [15]:
# Função para encontrar a votação mais equilibrada em termos de "Sim" e "Não"
def encontrar_votacao_equilibrada(df):
    votacoes_unicas = df['id_votacao'].unique()
    votacao_equilibrada = None
    diferenca_minima = float('inf')
    
    # Iterar sobre cada votação e calcular a diferença entre "Sim" e "Não"
    for id_votacao in votacoes_unicas:
        votos_na_votacao = df[df['id_votacao'] == id_votacao]
        votos_sim = len(votos_na_votacao[votos_na_votacao['voto'] == 'Sim'])
        votos_nao = len(votos_na_votacao[votos_na_votacao['voto'] == 'Não'])
        diferenca = abs(votos_sim - votos_nao)
        
        # Selecionar a votação com a menor diferença entre "Sim" e "Não"
        if diferenca < diferenca_minima:
            diferenca_minima = diferenca
            votacao_equilibrada = id_votacao
            
    return votacao_equilibrada

In [16]:
# Função para selecionar aleatoriamente X/2 deputados que votaram "Sim" e X/2 que votaram "Não"
def selecionar_deputados(df, votacao_id, X):
    # Filtrar a votação selecionada
    votacao_selecionada = df[df['id_votacao'] == votacao_id]
    
    # Deputados que votaram "Sim" e "Não"
    deputados_sim = votacao_selecionada[votacao_selecionada['voto'] == 'Sim']['id_deputado'].unique()
    deputados_nao = votacao_selecionada[votacao_selecionada['voto'] == 'Não']['id_deputado'].unique()
    
    # Selecionar X/2 deputados aleatoriamente de cada grupo
    deputados_selecionados_sim = np.random.choice(deputados_sim, size=X//2, replace=False)
    deputados_selecionados_nao = np.random.choice(deputados_nao, size=X//2, replace=False)
    
    # Combinar os deputados selecionados
    deputados_selecionados = np.concatenate((deputados_selecionados_sim, deputados_selecionados_nao))
    
    return deputados_selecionados

In [17]:
# Função para calcular a tabela de votos similares entre os deputados
def calcular_tabela_similaridade(df_votos, deputados):
    # Criar uma tabela (dataframe) com os deputados nas linhas e colunas
    tabela_similaridade = pd.DataFrame(0, index=deputados, columns=deputados)
    
    # Iterar sobre as votações e contar os votos similares
    votacoes_unicas = df_votos['id_votacao'].unique()
    for id_votacao in votacoes_unicas:
        votacao_atual = df_votos[df_votos['id_votacao'] == id_votacao]
        for i, deputado_1 in enumerate(deputados):
            for deputado_2 in deputados[i + 1:]:
                voto_1 = votacao_atual[votacao_atual['id_deputado'] == deputado_1]['voto'].values
                voto_2 = votacao_atual[votacao_atual['id_deputado'] == deputado_2]['voto'].values
                
                # Verificar se ambos votaram e se os votos são iguais
                if len(voto_1) > 0 and len(voto_2) > 0 and voto_1[0] == voto_2[0]:
                    tabela_similaridade.loc[deputado_1, deputado_2] += 1
                    tabela_similaridade.loc[deputado_2, deputado_1] += 1  # Simetria
    
    return tabela_similaridade

In [18]:
# Função para juntar os grafos gerados
def merge_graphs(graphs):
    print("[MERGE] Iniciando junção de grafos...")
    G = nx.Graph()
    for g_idx, g in enumerate(graphs):
        for deputado_1, deputado_2, data in g.edges(data=True):
            if not G.has_edge(deputado_1, deputado_2):
                G.add_edge(deputado_1, deputado_2, weight=data['weight'])
            else:
                G[deputado_1][deputado_2]['weight'] += data['weight']
        print(f"[MERGE] Concluído o grafo {g_idx + 1}/{len(graphs)}.")
    print("[MERGE] Junção de todos os grafos concluída.")
    return G

In [20]:
# Função para otimizar o processo com paralelismo e monitorar o progresso
def otimizar_com_paralelismo(df_votos, tipo_voto, idx_total_votacao):
    print(f"[{tipo_voto}] Iniciando otimização com paralelismo...")
    total_votacoes = len(df_votos['id_votacao'].unique())
    df_chunks = np.array_split(df_votos, cpu_count())  # Dividir em partes para paralelismo
    
    # Processar em paralelo
    with Pool(cpu_count()) as p:
        grafos = p.starmap(processar_votacoes, [(chunk, tipo_voto, total_votacoes, idx_total_votacao) for chunk in df_chunks])
    
    print(f"[{tipo_voto}] Otimização com paralelismo concluída.")
    return merge_graphs(grafos), total_votacoes

In [21]:
# Função para processar o conjunto de dados sem paralelismo
def processar_dados_sem_paralelismo(df_votos, tipo_voto, idx_total_votacao):
    print(f"[{tipo_voto}] Iniciando processamento sem paralelismo...")
    G = processar_votacoes(df_votos, tipo_voto, len(df_votos['id_votacao'].unique()), idx_total_votacao)
    return G, len(df_votos['id_votacao'].unique())

In [22]:
# Encontrar a votação mais equilibrada
votacao_equilibrada_id = encontrar_votacao_equilibrada(analise_votacao_filt)

# Selecionar aleatoriamente os deputados com base na votação mais equilibrada
deputados_selecionados = selecionar_deputados(analise_votacao_filt, votacao_equilibrada_id, n_deputados)

# Filtrar os dados para pegar apenas os deputados selecionados
analise_votacao_filt_dep = analise_votacao_filt[analise_votacao_filt['id_deputado'].isin(deputados_selecionados)]

In [23]:
# Criar subconjuntos do dataset com base nos votos
votos_sim = analise_votacao_filt_dep[analise_votacao_filt_dep['voto'] == 'Sim']
votos_nao = analise_votacao_filt_dep[analise_votacao_filt_dep['voto'] == 'Não']
votos_outros = analise_votacao_filt_dep[~analise_votacao_filt_dep['voto'].isin(['Sim', 'Não'])]

In [24]:
# Controlar o índice global de votações
total_votacoes_acumuladas = 0

# Processar separadamente cada subconjunto de votos
print("Iniciando processamento de votos SIM...")
start_sim = time.time()
G_sim, total_votacoes_sim = processar_dados_sem_paralelismo(votos_sim, 'Sim', total_votacoes_acumuladas)
total_votacoes_acumuladas += total_votacoes_sim
print(f"Tempo total para processar votos SIM: {time.time() - start_sim:.2f}s\n")

print("Iniciando processamento de votos NÃO...")
start_nao = time.time()
G_nao, total_votacoes_nao = processar_dados_sem_paralelismo(votos_nao, 'Não', total_votacoes_acumuladas)
total_votacoes_acumuladas += total_votacoes_nao
print(f"Tempo total para processar votos NÃO: {time.time() - start_nao:.2f}s\n")

print("Iniciando processamento de votos OUTROS...")
start_outros = time.time()
G_outros, total_votacoes_outros = processar_dados_sem_paralelismo(votos_outros, 'Outros', total_votacoes_acumuladas)
total_votacoes_acumuladas += total_votacoes_outros
print(f"Tempo total para processar votos OUTROS: {time.time() - start_outros:.2f}s\n")

# Mesclar os grafos gerados
G_final = merge_graphs([G_sim, G_nao, G_outros])

# Normalizar os pesos das arestas entre 0 e 1
max_peso = max(nx.get_edge_attributes(G_final, 'weight').values())
for (deputado_1, deputado_2, dados) in G_final.edges(data=True):
    G_final[deputado_1][deputado_2]['weight'] /= max_peso

# Exportar o grafo com as informações de nome, partido e estado
exportar_grafo_gml(G_final, "grafo_deputados_atualizado.gml")

Iniciando processamento de votos SIM...
[Sim] Iniciando processamento sem paralelismo...
[Sim] Iniciando processamento de 36210 votações...
[Sim] Votação 1/23 (ID: 2309053105.0) concluída. Total de comparações feitas: 6685420
[Sim] Votação 2/23 (ID: 232490052.0) concluída. Total de comparações feitas: 6971116
[Sim] Votação 3/23 (ID: 231912970.0) concluída. Total de comparações feitas: 7786966
[Sim] Votação 4/23 (ID: 231130063.0) concluída. Total de comparações feitas: 9455710
[Sim] Votação 5/23 (ID: 231130060.0) concluída. Total de comparações feitas: 10445314
[Sim] Votação 6/23 (ID: 231294892.0) concluída. Total de comparações feitas: 15508906
[Sim] Votação 7/23 (ID: 232079957.0) concluída. Total de comparações feitas: 15587491
[Sim] Votação 8/23 (ID: 1555470319.0) concluída. Total de comparações feitas: 16254283
[Sim] Votação 9/23 (ID: 231373462.0) concluída. Total de comparações feitas: 17770463
[Sim] Votação 10/23 (ID: 232026666.0) concluída. Total de comparações feitas: 18652127
[

In [25]:
# Calcular a tabela de votos similares entre os deputados
tabela_similaridade = calcular_tabela_similaridade(analise_votacao_filt_dep, deputados_selecionados)

# Exibir a tabela de similaridade
print("\nTabela de similaridade entre deputados:")
tabela_similaridade


Tabela de similaridade entre deputados:


,178836.0,204375.0,204508.0,205550.0,204436.0,197438.0,122158.0,160758.0,171623.0,204404.0,...,178990.0,178886.0,204553.0,204548.0,160592.0,204518.0,204573.0,178844.0,178866.0,139285.0
178836.0,0,3,7,9,2,10,2,5,3,6,...,5,9,7,7,4,8,5,4,5,6
204375.0,3,0,1,7,3,7,4,7,6,7,...,4,4,0,4,0,3,5,3,0,1
204508.0,7,1,0,4,2,7,2,6,3,8,...,5,6,10,6,7,6,4,4,6,5
205550.0,9,7,4,0,2,11,7,9,8,6,...,8,6,8,10,8,6,6,8,8,6
204436.0,2,3,2,2,0,5,2,5,5,5,...,5,2,0,1,1,0,4,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204518.0,8,3,6,6,0,8,4,7,4,5,...,5,7,8,5,5,0,3,6,4,3
204573.0,5,5,4,6,4,9,5,6,10,14,...,13,10,14,6,12,3,0,9,9,15
178844.0,4,3,4,8,3,8,3,8,6,8,...,12,5,12,5,10,6,9,0,10,6
178866.0,5,0,6,8,0,2,1,4,2,5,...,9,5,16,6,13,4,9,10,0,9


In [164]:
def votacoes_juntas(df_votos, deputado_1, deputado_2):
    # Filtrar as votações em que ambos os deputados participaram
    votacoes_deputado_1 = df_votos[df_votos['id_deputado'] == deputado_1]['id_votacao'].unique()
    votacoes_deputado_2 = df_votos[df_votos['id_deputado'] == deputado_2]['id_votacao'].unique()
    
    # Encontrar as votações em que ambos participaram
    votacoes_em_comum = np.intersect1d(votacoes_deputado_1, votacoes_deputado_2)
    
    # Verificar em quais dessas votações eles votaram da mesma forma
    votacoes_iguais = []
    for id_votacao in votacoes_em_comum:
        voto_1 = df_votos[(df_votos['id_votacao'] == id_votacao) & (df_votos['id_deputado'] == deputado_1)]['voto'].values[0]
        voto_2 = df_votos[(df_votos['id_votacao'] == id_votacao) & (df_votos['id_deputado'] == deputado_2)]['voto'].values[0]
        if voto_1 == voto_2:
            votacoes_iguais.append(id_votacao)
    
    return votacoes_iguais

In [165]:
def ranking_similaridade(tabela_similaridade):
    # Inicializar uma lista para armazenar os pares e os votos similares
    pares_similaridade = []
    
    # Iterar sobre as linhas e colunas da tabela de similaridade
    for deputado_1 in tabela_similaridade.index:
        for deputado_2 in tabela_similaridade.columns:
            if deputado_1 != deputado_2:  # Ignorar o par com o mesmo deputado
                # Adicionar o par e o número de votos similares à lista
                pares_similaridade.append((deputado_1, deputado_2, tabela_similaridade.loc[deputado_1, deputado_2]))
    
    # Transformar a lista em um DataFrame para facilitar a manipulação
    df_ranking = pd.DataFrame(pares_similaridade, columns=['Deputado 1', 'Deputado 2', 'Votos Similares'])
    
    # Remover duplicatas, pois (Deputado 1, Deputado 2) é o mesmo que (Deputado 2, Deputado 1)
    df_ranking = df_ranking[df_ranking['Deputado 1'] < df_ranking['Deputado 2']]
    
    # Ordenar o DataFrame pelo número de votos similares (do menor para o maior)
    df_ranking = df_ranking.sort_values(by='Votos Similares')
    
    return df_ranking

In [166]:
# Gerar o ranking dos pares de deputados com menos até mais votos em comum
df_ranking = ranking_similaridade(tabela_similaridade)

# Exibir o ranking
print("Ranking dos pares de deputados com base no número de votos similares:")
print(df_ranking)

Ranking dos pares de deputados com base no número de votos similares:
      Deputado 1  Deputado 2  Votos Similares
5361     73801.0    161550.0                0
9720     73482.0    178956.0                0
8730    160508.0    178956.0                0
8752    160508.0    204430.0                0
6271     74459.0    204354.0                0
...          ...         ...              ...
1406    204365.0    204461.0               22
5021    204383.0    204530.0               22
6188     76874.0    204383.0               22
8098     74376.0    160575.0               23
6209     76874.0    204530.0               23

[4950 rows x 3 columns]


In [167]:
deputado_1 = 204509.0
deputado_2 = 112437.0

# Encontrar as votações onde deputado_1 e deputado_2 votaram da mesma forma
votacoes_em_comum = votacoes_juntas(analise_votacao_filt_dep, deputado_1, deputado_2)

print(f"Deputado {deputado_1} e Deputado {deputado_2} votaram juntos nas seguintes votações: {votacoes_em_comum}")

Deputado 204509.0 e Deputado 112437.0 votaram juntos nas seguintes votações: []


In [168]:
def obter_voto(df_votos, id_votacao, id_deputado):
    # Filtrar o dataframe para a votação e o deputado específicos
    voto_deputado = df_votos[(df_votos['id_votacao'] == id_votacao) & (df_votos['id_deputado'] == id_deputado)]
    
    # Verificar se existe um voto para esse deputado na votação
    if not voto_deputado.empty:
        return voto_deputado['voto'].values[0]  # Retorna o voto do deputado
    else:
        return None  # Retorna None se não houver voto registrado

In [169]:
id_votacao = 2694699.0  # Substitua pelo ID da votação
id_deputado = 112437.0  # Deputado que você deseja verificar

# Encontrar o voto do deputado na votação
voto = obter_voto(analise_votacao_filt_dep, id_votacao, id_deputado)

if voto:
    print(f"O Deputado {id_deputado} votou '{voto}' na votação {id_votacao}.")
else:
    print(f"O Deputado {id_deputado} não participou da votação {id_votacao}.")

O Deputado 112437.0 não participou da votação 2694699.0.
